In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gzip
import numpy as np
import tensorflow as tf
from typing import Tuple

Let's start by including the code that gets the data and model, which you should be very familiar with by now.

In [6]:
labels_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
  }


def read_images(path: str, image_size: int, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  buf = f.read(image_size * image_size * num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
  data = data.reshape(num_items, image_size, image_size)
  return data


def read_labels(path: str, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  f.read(8)
  buf = f.read(num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  data = data.reshape(num_items)
  return data


def get_data(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
  image_size = 28
  num_train = 60000
  num_test = 10000

  training_images = read_images('data/FashionMNIST/raw/train-images-idx3-ubyte.gz', image_size, num_train)
  test_images = read_images('data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz', image_size, num_test)
  training_labels = read_labels('data/FashionMNIST/raw/train-labels-idx1-ubyte.gz', num_train)
  test_labels = read_labels('data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz', num_test)

  # (training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
  test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

  train_dataset = train_dataset.batch(batch_size).shuffle(500)
  test_dataset = test_dataset.batch(batch_size).shuffle(500)

  return (train_dataset, test_dataset)


def get_model() -> tf.keras.Model:
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  return model

In order to make a prediction, we need to pass some data to the model, and do a single forward pass through the network to get the prediction. If the code below is unclear to you, make sure you go back to module 1, where we explain it in detail. Remember that, unlike during testing, we don't need to call the loss function because we're no longer interested in evaluating how well the model is doing. Instead, we call `softmax` to convert the values of the output vector into values between 0 and 1, and then get the `argmax` of that vector to get the predicted label index.

Similarly to the training and testing sections, once we're done with debugging, we can add a `@tf.function` decorator to get the performance benefits of graph execution.

In [7]:
@tf.function
def predict(model: tf.keras.Model, X: np.ndarray) -> tf.Tensor:
  y_prime = model(X, training=False)
  probabilities = tf.nn.softmax(y_prime, axis=1)
  predicted_indices = tf.math.argmax(input=probabilities, axis=1)
  return predicted_indices

Typically, during inference, we give the model input data that it hasn't seen before. In this sample, for simplicity, we input the first 3 images of the test dataset instead. 

In [8]:
batch_size = 64

model = get_model()
model.load_weights('outputs/weights')

(_, test_dataset) = get_data(batch_size)
(X_batch, actual_index_batch) = next(test_dataset.as_numpy_iterator())
X = X_batch[0:3, :, :]
actual_indices = actual_index_batch[0:3]

predicted_indices = predict(model, X)

print('\nPredicting:')
for (actual_index, predicted_index) in zip(actual_indices, predicted_indices):
  actual_name = labels_map[actual_index]
  predicted_name = labels_map[predicted_index.numpy()]
  print(f'Actual: {actual_name}, Predicted: {predicted_name}')


Predicting:
Actual: Sandal, Predicted: Sandal
Actual: T-Shirt, Predicted: T-Shirt
Actual: Bag, Predicted: Bag
